In [1]:
import pandas as pd
import numpy as np
import tinkoffpy as tf
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
from datetime import datetime, timedelta, date
import datetime

%matplotlib inline

now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

tf.auth_by_credentials(username='r.yunisov', password='Htyfn2002Htyfn')



/Users/r.yunisov/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df = tf.gp_to_df("""
SELECT *, ROUND((cum_sum::NUMERIC / total_clients::NUMERIC) * 100, 2) AS prop
FROM (SELECT *,
             SUM(sessions)
             OVER (PARTITION BY entity_registration_week ORDER BY date_diff ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_sum,
             SUM(clients) OVER (PARTITION BY entity_registration_week)                                                       AS total_clients
      FROM (SELECT entity_registration_week,
                   date_diff,
                   COUNT(party_rk_regist)          AS clients,
                   COUNT(party_rk_session) AS sessions
            FROM usr_wrk.first_session_after_registr
            GROUP BY entity_registration_week,
                     date_diff) t) tt
WHERE date_diff IS NOT NULL
""", gp_service = 'sme')

df.head()

2023-12-07 12:50:12 - WARNING - Library version is outdated, you have: 1.0.49
actual version: 1.0.53, please, update library.
2023-12-07 12:50:53 - INFO - Query executed successfully. Result saved to DataFrame.


,entity_registration_week,date_diff,clients,sessions,cum_sum,total_clients,prop
0,2023-10-30 00:00:00+03:00,36,1,1,5818.0,9372.0,62.08
1,2023-09-18 00:00:00+03:00,56,4,4,5832.0,9010.0,64.73
2,2023-10-09 00:00:00+03:00,55,4,4,6073.0,9426.0,64.43
3,2023-09-04 00:00:00+03:00,87,12,12,5509.0,8312.0,66.28
4,2023-09-11 00:00:00+03:00,46,13,13,5435.0,8593.0,63.25


In [4]:
fig_vintage = px.line(df.groupby(['entity_registration_week', 'date_diff'], as_index=False)['prop'].agg('max').query("date_diff <= 50"), x='date_diff', y='prop', 
                      markers=True, 
                      width=1800, 
                      height=800,
                      color='entity_registration_week')


fig_vintage.show()